# push_to_hub

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     |████████████████████████████████| 312 kB 4.0 MB/s 
     |████████████████████████████████| 3.8 MB 40.6 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 134 kB 47.0 MB/s 
     |████████████████████████████████| 1.1 MB 45.2 MB/s 
     |████████████████████████████████| 212 kB 58.0 MB/s 
     |████████████████████████████████| 127 kB 60.5 MB/s 
     |████████████████████████████████| 271 kB 56.8 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 59.1 MB/s 
     |████████████████████████████████| 6.5 MB 43.4 MB/s 
     |████████████████████████████████| 596 kB 63.0 MB/s 
     |████████████████████████████████| 895 kB 68.7 MB/s 
     |████████████████████████████████| 1.2 MB 52.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

You will need to setup git, adapt your email and name in the following cell.

In [15]:
!git config --global user.email "your_email_here"
!git config --global user.name "github_username_here"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [16]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [17]:
from datasets import load_dataset
from transformers import AutoTokenizer, BertTokenizerFast
from transformers import AutoModelForQuestionAnswering, BertForQuestionAnswering
import collections
import numpy as np
from datasets import load_metric
from tqdm.auto import tqdm
from transformers import TrainingArguments
from transformers import Trainer

In [22]:
def push_model_to_hub_wrap(model_checkpoint,train_path,dev_path):

    data_files = {"train": train_path, "validation": dev_path}
    raw_datasets = load_dataset("json", data_files=data_files, field='data')

    raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

    tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

    context = raw_datasets["train"][0]["context"]
    question = raw_datasets["train"][0]["question"]

    inputs = tokenizer(question, context)
    tokenizer.decode(inputs["input_ids"])

    train_set = raw_datasets["train"]
    val_set = raw_datasets["validation"]

    max_length = 384
    stride = 128


    def preprocess_training_examples(examples):
        questions = [q.strip() for q in examples["question"]]
        inputs = tokenizer(
            questions,
            examples["context"],
            max_length=max_length,
            truncation="only_second",
            stride=stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        offset_mapping = inputs.pop("offset_mapping")
        sample_map = inputs.pop("overflow_to_sample_mapping")
        answers = examples["answers"]
        start_positions = []
        end_positions = []

        for i, offset in enumerate(offset_mapping):
            sample_idx = sample_map[i]
            answer = answers[sample_idx]
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
            sequence_ids = inputs.sequence_ids(i)

            # Find the start and end of the context
            idx = 0
            while sequence_ids[idx] != 1:
                idx += 1
            context_start = idx
            while sequence_ids[idx] == 1:
                idx += 1
            context_end = idx - 1

            # If the answer is not fully inside the context, label is (0, 0)
            if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Otherwise it's the start and end token positions
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)

        inputs["start_positions"] = start_positions
        inputs["end_positions"] = end_positions
        return inputs


    train_dataset = train_set.map(
        preprocess_training_examples,
        batched=True,
        remove_columns=train_set.column_names,
    )


    def preprocess_validation_examples(examples):
        questions = [q.strip() for q in examples["question"]]
        inputs = tokenizer(
            questions,
            examples["context"],
            max_length=max_length,
            truncation="only_second",
            stride=stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        sample_map = inputs.pop("overflow_to_sample_mapping")
        example_ids = []

        for i in range(len(inputs["input_ids"])):
            sample_idx = sample_map[i]
            example_ids.append(examples["id"][sample_idx])

            sequence_ids = inputs.sequence_ids(i)
            offset = inputs["offset_mapping"][i]
            inputs["offset_mapping"][i] = [
                o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
            ]

        inputs["example_id"] = example_ids
        return inputs


    validation_dataset = val_set.map(
        preprocess_validation_examples,
        batched=True,
        remove_columns=val_set.column_names,
    )

    metric = load_metric("squad")

    n_best = 20
    max_answer_length = 30


    def compute_metrics(start_logits, end_logits, features, examples):
        example_to_features = collections.defaultdict(list)
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        predicted_answers = []
        for example in tqdm(examples):
            example_id = example["id"]
            context = example["context"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1: -n_best - 1: -1].tolist()
                end_indexes = np.argsort(end_logit)[-1: -n_best - 1: -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                                end_index < start_index
                                or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answer = {
                            "text": context[offsets[start_index][0]: offsets[end_index][1]],
                            "logit_score": start_logit[start_index] + end_logit[end_index],
                        }
                        answers.append(answer)

            # Select the answer with the best score
            if len(answers) > 0:
                best_answer = max(answers, key=lambda x: x["logit_score"])
                predicted_answers.append(
                    {"id": example_id, "prediction_text": best_answer["text"]}
                )
            else:
                predicted_answers.append({"id": example_id, "prediction_text": ""})

        theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
        return metric.compute(predictions=predicted_answers, references=theoretical_answers)


    model = BertForQuestionAnswering.from_pretrained(model_checkpoint)

    args = TrainingArguments(
        "hebert-finetuned-hebrew-squad",
        evaluation_strategy="no",
        save_strategy="epoch",
        learning_rate=2e-5,
        num_train_epochs=15,
        weight_decay=0.01,
        fp16=False,
        push_to_hub=True,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        tokenizer=tokenizer,
    )


    trainer.push_to_hub(commit_message="Training complete")

In [23]:
our_model_name = "hebert-finetuned-hebrew-squad"
name_to_hub="hebert_finetuned_hebrew_squad" #need to be different from our_model_name

# Please choose dataset:
train_path = "train.json"
dev_path = "validation.json"
push_model_to_hub_wrap(model_checkpoint=our_model_name,train_path=train_path,dev_path=dev_path)


Using custom data configuration default-a36d603f9f16ca0a
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-a36d603f9f16ca0a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-a36d603f9f16ca0a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-233f11e4e1652339.arrow
Didn't find file hebert-finetuned-heb-squad/added_tokens.json. We won't load it.
loading file hebert-finetuned-heb-squad/vocab.txt
loading file hebert-finetuned-heb-squad/tokenizer.json
loading file None
loading file hebert-finetuned-heb-squad/special_tokens_map.json
loading file hebert-finetuned-heb-squad/tokenizer_config.json


  0%|          | 0/53 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

loading configuration file hebert-finetuned-heb-squad/config.json
Model config BertConfig {
  "_name_or_path": "avichr/heBERT",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "total_flos": 6997313242916978688,
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file hebert-finetuned-heb-squad/pytorch_model.bin
All model checkpoint weights were used when initializing BertForQuestionAnswering.

All the weights of BertForQuestionAnswering were init

Upload file pytorch_model.bin:   0%|          | 3.34k/415M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 2.98k/2.98k [00:00<?, ?B/s]

To https://huggingface.co/MatanBenChorin/hebert-finetuned-hebrew-squad
   0f94136..c66dcaa  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
To https://huggingface.co/MatanBenChorin/hebert-finetuned-hebrew-squad
   c66dcaa..3fa15e3  main -> main

